In [16]:
import tensorflow as tf
from tensorflow.keras import layers

# Gradient Clipping (add this to your optimizer)
optimizer = tf.keras.optimizers.Adam(clipnorm=1.0) 

# Encoder
encoder_input = layers.Input(shape=(164, 1))
x = layers.Conv1D(64, 9, activation="relu", padding="same")(encoder_input)
x = layers.BatchNormalization()(x)  # Normalization
x = layers.MaxPooling1D(2, padding="same")(x)
x = layers.Conv1D(32, 7, activation="relu", padding="same")(x)
x = layers.BatchNormalization()(x) 
x = layers.MaxPooling1D(2, padding="same")(x)
x = layers.Conv1D(16, 5, activation="relu", padding="same")(x)
x = layers.BatchNormalization()(x) 
encoded = layers.MaxPooling1D(2, padding="same", name="encoder")(x)

# Decoder
x = layers.Conv1D(16, 5, activation="relu", padding="same")(encoded)
x = layers.BatchNormalization()(x) 
x = layers.UpSampling1D(2)(x)
x = layers.Conv1D(32, 7, activation="relu", padding="same")(x)
x = layers.BatchNormalization()(x) 
x = layers.UpSampling1D(2)(x)
x = layers.Conv1D(64, 9, activation="relu", padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling1D(2)(x)
decoded = layers.Conv1D(1, 7, activation="sigmoid", padding="same", 
                        kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)  # L2 regularization

# Autoencoder model
autoencoder = tf.keras.Model(encoder_input, decoded)
autoencoder.compile(optimizer=optimizer, loss='binary_crossentropy') 

In [17]:
from src.dataset import load_dataset, Species, Modification

dataset = load_dataset(Species.human, Modification.psi)

In [18]:
from src.features.encodings import binary

encoder = binary.Encoder()

In [24]:
encoded_samples = encoder.fit_transform(dataset.samples).values
targets = dataset.targets

In [25]:
targets.values

0       1
1       1
2       1
3       1
4       1
       ..
7909    0
7910    0
7911    0
7912    0
7913    0
Name: 1, Length: 7914, dtype: int64

In [26]:
autoencoder.fit(encoded_samples, targets,
                epochs=50,
                batch_size=256,
                shuffle=True)

Epoch 1/50


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 1), output.shape=(None, 168)